<a href="https://colab.research.google.com/github/NishitSinha/Personalized-Real-Estate-Agent/blob/main/HomeMatch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%env OPENAI_API_KEY = voc-113248041712667745788626912f17c722c20.17083193
%env OPENAI_API_BASE = https://openai.vocareum.com/v1

env: OPENAI_API_KEY=voc-113248041712667745788626912f17c722c20.17083193
env: OPENAI_API_BASE=https://openai.vocareum.com/v1


## Installing all the dependent packages

In [ ]:
!pip install -U -q openai langchain langchain-openai langchain-chroma langchain-core langchain-text-splitters chromadb sentence-transformers

## Importing libraries

In [149]:
import os
import json
import textwrap
import pandas as pd
from pydantic import BaseModel
from typing import Optional
from openai import OpenAI
from langchain_openai import OpenAIEmbeddings
from langchain_chroma import Chroma
from langchain_core.documents import Document

## Defining function required to use multile times

In [ ]:
api_key = os.getenv("OPENAI_API_KEY")
base_url = os.getenv("OPENAI_API_BASE")
chatbot = OpenAI(
    api_key = api_key,
    base_url = base_url
)
print("Chatgpt inicated")

Chatgpt inicated


In [ ]:
def resp(prompt):
  responce = chatbot.chat.completions.create(
      model = "gpt-3.5-turbo",
      messages = [{"role":"user" , "content":prompt}],
      temperature = 1.0
  )
  ans = responce.choices[0].message.content.strip()
  return ans

## Creating Real Estate Housing CSV

In [ ]:
prompt_csv = """
Generate 20 realistic real estate listings
In cvs format and must follow these ristrictions
1: No commas in int(numberic) fields
2: Description should not contain commas
3: All non int(numberic) fields must be in double quotes
4: Don't double quotes the headers
5: No spaces before and after comma


Headers:
1: Neighborhood : str
2: Price : int(natural numbers)
3: Bedroom : int(natural numbers)
4: Bathroom : int(natural numbers)
5: House Space : int(natural numbers)
6: Description : str
"""

In [ ]:
csv_data = resp(prompt_csv)
print(csv_data)

Neighborhood,Price,Bedroom,Bathroom,House Space,Description
"Brooklyn",500000,3,2,1500,"Cozy 3 bedroom 2 bathroom home in the heart of Brooklyn"
"Manhattan",1000000,2,1,1200,"Beautiful 2 bedroom 1 bathroom apartment with stunning city views"
"Queens",700000,4,3,2000,"Spacious 4 bedroom 3 bathroom house with a large backyard"
"Bronx",400000,2,1,1000,"Charming 2 bedroom 1 bathroom starter home in a quiet neighborhood"
"Staten Island",600000,3,2,1800,"Renovated 3 bedroom 2 bathroom colonial style house with a modern kitchen"
"Harlem",800000,5,3,2500,"Large 5 bedroom 3 bathroom townhouse with original architectural details"
"Upper East Side",1500000,4,4,3000,"Luxurious 4 bedroom 4 bathroom penthouse with panoramic city views"
"Chelsea",1200000,3,2,1800,"Modern 3 bedroom 2 bathroom condo with high-end finishes"
"Williamsburg",900000,2,2,1600,"Sleek 2 bedroom 2 bathroom loft with exposed brick walls and industrial accents"
"Greenwich Village",1100000,1,1,1000,"Cozy 1 bedroom 1 bathroom apart

In [ ]:
fd = open("houselistings.csv" , "w")
fd.write(csv_data)
fd.close()

In [ ]:
df = pd.read_csv("houselistings.csv")
print(df.head())

    Neighborhood    Price  Bedroom  Bathroom  House Space  \
0       Brooklyn   500000        3         2         1500   
1      Manhattan  1000000        2         1         1200   
2         Queens   700000        4         3         2000   
3          Bronx   400000        2         1         1000   
4  Staten Island   600000        3         2         1800   

                                         Description  
0  Cozy 3 bedroom 2 bathroom home in the heart of...  
1  Beautiful 2 bedroom 1 bathroom apartment with ...  
2  Spacious 4 bedroom 3 bathroom house with a lar...  
3  Charming 2 bedroom 1 bathroom starter home in ...  
4  Renovated 3 bedroom 2 bathroom colonial style ...  


## Storing Listing data into ChromaDB

Converting Rows to text

In [ ]:
doc = []
for i, data in df.iterrows():
  text = data['Neighborhood'] + str(data['Price']) + str(data['Bedroom']) + "Bedrooms" + str(data['Bathroom']) + "Bathrooms" + str(data['House Space']) + "sqft" + data['Description']
  doc.append(Document(
      page_content = text,
      metadata = data.to_dict()
      )
  )

Embedding and Storing in database

In [ ]:
embedder = OpenAIEmbeddings(
    model = "text-embedding-3-small",
    api_key = api_key,
    openai_api_base = base_url
)
print("Inicating OpenAIEmbedding")

Inicating OpenAIEmbedding


In [ ]:
db = Chroma.from_documents(
    documents = doc,
    embedding = embedder,
    collection_name = "Houselisting_vectordb"
)
print("Croma Vector DB created")

Croma Vector DB created


## Semantic Search

Taking User Input

In [ ]:
class Buyerpref(BaseModel):
  pref: Optional[str] = None
  budget: Optional[int] = None
  bedroom: Optional[int] = None
  bathroom: Optional[int] = None
  size: Optional[int] = None

In [ ]:
ex = "I need a house near Brooklyn having 2 Bedrooms and 2 Bathrooms under $750000 and must have more then 1000sqft space"
print("Preference Example :~ ", ex)
user_io = input("Enter Your Priferences Here :~ ")

Preference Example :~  I need a house near Brooklyn having 2 Bedrooms and 2 Bathrooms under $750000 and must have more then 1000sqft space
Enter Your Priferences Here :~  


In [ ]:
if not user_io.strip():
  user_io = ex

In [ ]:
prompt_json = """
Extract JSON format :
{{
  "pref": string|null
  "budget": int|null
  "bedroom": int|null
  "bathroom": int|null
  "size": int|null
}}

from following text :
""" + user_io

In [ ]:
user_pref = resp(prompt_json)
pref = Buyerpref(**json.loads(user_pref))
print(pref)

pref='Brooklyn' budget=750000 bedroom=2 bathroom=2 size=1000


Taking input for missing values

In [ ]:
mis = [i for i , j in pref.model_dump().items() if j is None and i != pref]
if mis:
  print("Please provide values for missing data:~ ")
  for i in mis:
    val = input("Enter value for "+ i + "(or leave blank) :~ ")
    if val.strip():
      try:
        setattr(pref, i , int(val))
      except:
        pass
print("Final Preference :~ ", pref.model_dump())

Final Preference :~  {'pref': 'Brooklyn', 'budget': 750000, 'bedroom': 2, 'bathroom': 2, 'size': 1000}


Semantic search in CromaDB


In [ ]:
prompt_search = " ".join([
    pref.pref or "",
    str(pref.budget) if pref.budget else "",
    str(pref.bedroom) + "Bedrooms" if pref.bedroom else "",
    str(pref.bathroom) + "Bathrooms" if pref.bathroom else "",
    str(pref.size) + "sqft" if pref.size else ""
    ])

In [143]:
similar_houses = db.similarity_search(prompt_search , k = 3)

## Augmenting Listing Descriptions

In [144]:
rec_house = []
for i in similar_houses:
  rec_house.append(i)

In [145]:
print("<-----------Top Matches---------------> \n")
for i, j in enumerate(rec_house):
  print("Recomended Home No. :~ " + str(i + 1) + "\n")
  for k, v in j.metadata.items():
    print(f"{k} : {v}")
  print("\n")
if len(rec_house) == 0:
  print("No Recomendation found in the listings")

<-----------Top Matches---------------> 

Recomended Home No. :~ 1

Description : Cozy 3 bedroom 2 bathroom home in the heart of Brooklyn
Bathroom : 2
Neighborhood : Brooklyn
House Space : 1500
Price : 500000
Bedroom : 3


Recomended Home No. :~ 2

Bathroom : 1
Description : Beautiful 2 bedroom 1 bathroom apartment with stunning city views
Price : 1000000
House Space : 1200
Neighborhood : Manhattan
Bedroom : 2


Recomended Home No. :~ 3

Bathroom : 2
Price : 900000
Bedroom : 2
House Space : 1600
Description : Sleek 2 bedroom 2 bathroom loft with exposed brick walls and industrial accents
Neighborhood : Williamsburg




In [150]:
if rec_house:
  best_rec = rec_house[0].metadata
  prompt_rec = f"""
  Create a personalized house recommendation using the buyers preference and the best recomended listing
  Make it look more tempting offer with a touch of casual notes like emotes or positives about neibourhood.
  User preference = {pref.model_dump()}
  Best Matching home = {json.dumps(best_rec)}
  """
  best_rec_desc = resp(prompt_rec)
  print("Personalized Recommedation :~ \n")
  print(textwrap.fill(best_rec_desc , width = 100))

Personalized Recommedation :~ 

Hey there! 🏡 I found the perfect cozy home for you in the heart of Brooklyn!   🌟 Description: Cozy 3
bedroom 2 bathroom home 🛏️ Bedroom: 3  🚽 Bathroom: 2  🏠 House Space: 1500 sqft  🏘️ Neighborhood:
Brooklyn   And the best part? It's listed below your budget at $500,000! This home offers more space
than you're looking for at 1500 sqft and an extra bedroom for versatility.   With a price like that,
this gem won't stay on the market for long! Don't miss out on this amazing opportunity to find your
dream home in Brooklyn! 🌆💫  Let me know if you'd like to schedule a viewing or if you have any
questions. Happy house hunting! 🏡🔑🌟


## For creating Submission files

In [151]:
fd = open("Listings.txt" , "w")
fd.write(csv_data)
fd.close()

In [155]:
import importlib
from importlib.metadata import version, PackageNotFoundError

PACKAGE_MAP = {
    "openai": "openai",
    "pandas": "pandas",
    "pydantic": "pydantic",
    "langchain": "langchain",
    "langchain_openai": "langchain-openai",
    "langchain_core": "langchain-core",
    "langchain_chroma": "langchain-chroma",
    "chromadb": "chromadb",
}

def get_verson(import_name , pip_name):
  try:
    importlib.import_module(import_name)
  except:
    return "Not Installed"
  try:
    return version(pip_name)
  except PackageNotFoundError:
    return "Version Not Found"
with open("Requirements.txt" , "w") as fd:
  for import_name , pip_name in PACKAGE_MAP.items():
    s = f"{import_name} : {get_verson(import_name , pip_name)}"
    print(s)
    fd.write(s.replace(" : ","==")+"\n")

openai : 2.8.1
pandas : 2.2.2
pydantic : 2.12.4
langchain : 1.0.8
langchain_openai : 1.0.3
langchain_core : 1.0.7
langchain_chroma : 1.0.0
chromadb : 1.3.5
